In [2]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
import scipy

import os

import numpy as np
from keras.preprocessing import image

In [11]:
VALIDATION_DIR = '/home/jack/Documents/data/rps/rps_val'
TRAINING_DIR = '/home/jack/Documents/data/rps/rps_train'
TEST_DIR = '/home/jack/Documents/data/rps/rps_pred'

In [12]:
# rock_dir = os.path.join(TRAINING_DIR, 'rock')
# paper_dir = os.path.join(TRAINING_DIR, 'paper')
# scissors_dir = os.path.join(TRAINING_DIR, 'scissors')

In [13]:
model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 15, 15, 128)      

In [14]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [15]:
training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical',
  batch_size=126
)

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    class_mode='categorical',
  batch_size=126
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


In [16]:
history = model.fit(train_generator, 
                    epochs=5, 
                    steps_per_epoch=20, 
                    validation_data = validation_generator, 
                    verbose = 1, 
                    validation_steps=3)

model.save("rps.h5")

Epoch 1/5
20/20 [==============================] - 160s 8s/step - loss: 1.4325 - acc: 0.3607 - val_loss: 1.0849 - val_acc: 0.3333
Epoch 2/5
20/20 [==============================] - 161s 8s/step - loss: 1.0735 - acc: 0.3964 - val_loss: 0.9283 - val_acc: 0.5565
Epoch 3/5
20/20 [==============================] - 157s 8s/step - loss: 1.0226 - acc: 0.4603 - val_loss: 1.2381 - val_acc: 0.5081
Epoch 4/5
20/20 [==============================] - 159s 8s/step - loss: 1.0158 - acc: 0.5254 - val_loss: 0.8101 - val_acc: 0.8495
Epoch 5/5
20/20 [==============================] - 163s 8s/step - loss: 0.8202 - acc: 0.6262 - val_loss: 0.5263 - val_acc: 0.7661


In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(150,150)
)

filenames = test_generator.filenames
nb_samples = len(filenames)

In [ ]:
classes = model.predict(test_generator,steps = 1 )

In [ ]:
# paper-rock-scissors
#[0.8 - 0.2 - 0.0]

In [ ]:
for i in range(nb_samples):
    
#     if classes[i].argmax() == 0:
#         prediction = 'paper'
#     elif classes[i].argmax() == 1:
#         prediction = 'rock'
#     else:
#         prediction = 'scissors'

#     print(f"{filenames[i]} -> {prediction}")
    print(f"{filenames[i].split('/')[1]} -> {classes[i]}")